In [ ]:
from custom_module import parse_yf_esg

# Введение

# Часть первая: Сбор данных

Для сбора данных нам необходимо для начала получить максимально возможное количество тикеров. Самая большая бесплатная база данных, которую мы нашли, это файл на сайте https://www.nasdaq.com/market-activity/stocks/screener
Далее необходимо собрать все необходимые данные: значения оценок esg рисков, прибыль по годам, цены на акции и прочие


Volodya huisos
